<a href="https://colab.research.google.com/github/sahandv/science_science/blob/master/cluster_labeling_tf_icf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# datapath = '/mnt/6016589416586D52/Users/z5204044/GoogleDrive/GoogleDrive/Data/' # Local
datapath = 'drive/My Drive/Data/' # Remote

In [ ]:
!rm -rf 'science_science'
username = "sahandv"#@param {type:"string"}
# password = ""#@param {type:"string"} 

!git clone https://github.com/$username/science_science.git
!ls

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install -r 'science_science/requirements.txt'
! pip install gensim==3.8.1

In [ ]:
import sys
import time
import gc
import collections
import json
import re
import os
import pprint
from random import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering, KMeans, SpectralClustering, AffinityPropagation
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from yellowbrick.cluster import KElbowVisualizer
import scipy.cluster.hierarchy as sch
from scipy import spatial,sparse,sign
from sklearn.feature_extraction.text import TfidfTransformer , TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from bokeh.io import push_notebook, show, output_notebook, output_file
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
from gensim.models import FastText as fasttext_gensim
from gensim.test.utils import get_tmpfile

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words("english"))
tqdm.pandas()

from science_science.sciosci.assets import keyword_assets as kw
from science_science.sciosci.assets import generic_assets as sci
from science_science.sciosci.assets import advanced_assets as aa

# from sciosci.assets import text_assets as kw
# from sciosci.assets import generic_assets as sci
# from sciosci.assets import advanced_assets as aa

In [ ]:
#@markdown Don't forget to set the "year_to" to a year higher than intended!
year_from = 1990#@param {type:"number"}
year_to = 2020#@param {type:"number"}
period = str(year_from)+'-'+str(year_to-1)
#@markdown File address for main WoS file:
# file_address = datapath+'Corpus/AI 4k/copyr_deflem_stopword_removed_thesaurus May 28/by period/n-gram by 6 repetition keywords/'+period+' abstract_title'
file_address = datapath+'Corpus/AI 4k/copyr_deflem_stopword_removed_thesaurus May 28/1990-2019/1990-2019 n-gram by 6 repetition keywords'
abstracts = pd.read_csv(file_address,names=['abstract'])
print('period:',period,'\n',abstracts)

In [ ]:
def get_abstract_keywords(corpus,keywords_wanted,max_df=0.9,max_features=None):
    cv=CountVectorizer(max_df=max_df,stop_words=stop_words, max_features=max_features, ngram_range=(1,1))
    X=cv.fit_transform(corpus)
    # get feature names
    feature_names=cv.get_feature_names()
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    tfidf_transformer.fit(X)
    keywords_tfidf = []
    keywords_sorted = []
    for doc in tqdm(corpus,total=len(corpus)):
        tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
        sorted_items=kw.sort_coo(tf_idf_vector.tocoo())
        keywords_sorted.append(sorted_items)
        keywords_tfidf.append(kw.extract_topn_from_vector(feature_names,sorted_items,keywords_wanted))
    return keywords_tfidf

def get_corpus_top_keywords(abstract_keywords_dict=None):
    if abstract_keywords_dict == None:
        print("keywords should be provided")
        return False
    terms = []
    values = []
    for doc in abstract_keywords_dict:
        if doc != None:
            terms = terms+list(doc.keys())
            values = values+list(doc.values())
    terms_df = pd.DataFrame({'terms':terms,'value':values}).groupby('terms').sum().sort_values('value',ascending=False)
    return terms_df

def find_max_item_value_in_all_cluster(haystack,needle,cluster_exception=None):
    max_val = 0
    max_index = None
    counter = 0
    for item in haystack:
        try:
            if item[needle]>max_val:
                if cluster_exception==None:
                    max_val = item[needle]
                    max_index = counter
                else:
                    if cluster_exception != counter:
                        max_val = item[needle] 
                        max_index = counter
        except:
            pass
        counter+=1

        if max_index!=None:
            row_max = haystack[max_index][list(haystack[max_index].keys())[0]] # Will give the maximum value (first item) of the row with max value of the needle. This gives us a perspective to see how this score compares to the max in the same row.
        else:
            row_max = 0
    # except:
        # row_max = None
    

    return max_val,row_max

In [ ]:
cluster_path = datapath+'Corpus/AI 4k/embeddings/clustering/k10/DEC/200,500,20/Doc2Vec patent_wos_ai corpus DEC 200,500,20 k10 labels'
clusters_df = pd.read_csv(cluster_path)
clusters_df.columns = ['clusters']
clusters_df

In [ ]:
clusters = clusters_df.groupby('clusters').groups
clusters.keys()

In [ ]:
# TF-IDF (CTF-ICF)
cluster_as_string = []
year_abstracts = pd.read_csv(file_address,names=['abstract'])['abstract']
clusters = clusters_df.groupby('clusters').groups
for key in clusters.keys():
    cluster_as_string.append(' '.join(year_abstracts[list(clusters[key])]))
cluster_keywords_tfidf = get_abstract_keywords(cluster_as_string,1000,max_df=0.8)

cluster_keywords = []
cluster_index = 0
for items in cluster_keywords_tfidf:
    items_tmp = []
    for item in items:
        max_data = find_max_item_value_in_all_cluster(cluster_keywords_tfidf,item,cluster_index)
        items_tmp.append(item+' ('+str(items[item])+' | '+str(max_data[0])+'/'+str(max_data[1])+')') # (item+' :'+str(items[item])+' / '+str( max of item in all other rows))
    cluster_keywords.append(items_tmp)
    cluster_index+=1
pd.DataFrame(cluster_keywords).to_csv(cluster_path+" keywords",index=False,header=False)

In [ ]:
# Get term cluster labels (just terms and not scores)
cluster_keywords_terms = []
cluster_keywords_scores = []
for item in cluster_keywords_tfidf:
    cluster_keywords_terms.append(list(item.keys()))
    cluster_keywords_scores.append(list(item.values()))

pd.DataFrame(cluster_keywords_terms).T.to_csv(cluster_path+" term_cluster",index=False)
pd.DataFrame(cluster_keywords_scores).T.to_csv(cluster_path+" term_score",index=False)

In [ ]:
# Get term frequencies for each period
terms = ' '.join(cluster_as_string).split()
terms = [x for x in terms if x not in list(stop_words)]
pd.DataFrame(terms,columns=['terms'])['terms'].value_counts().to_csv(cluster_path+"  frequency",header=False)